<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Feedback" data-toc-modified-id="Feedback-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feedback</a></span><ul class="toc-item"><li><span><a href="#returning-the-clue-as-a-string-vs-a-list" data-toc-modified-id="returning-the-clue-as-a-string-vs-a-list-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>returning the clue as a string vs a list</a></span></li><li><span><a href="#Evaluating-the-time-to-correct-the-wordle-feedback" data-toc-modified-id="Evaluating-the-time-to-correct-the-wordle-feedback-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Evaluating the time to correct the wordle feedback</a></span></li><li><span><a href="#Speeding-up-the-feedback" data-toc-modified-id="Speeding-up-the-feedback-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Speeding up the feedback</a></span></li></ul></li><li><span><a href="#Evaluating-the-next-guess-analysis" data-toc-modified-id="Evaluating-the-next-guess-analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evaluating the next guess analysis</a></span><ul class="toc-item"><li><span><a href="#Time-to-get-feedback" data-toc-modified-id="Time-to-get-feedback-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Time to get feedback</a></span></li><li><span><a href="#Time-to-create-the-whole-analysis-dataframe" data-toc-modified-id="Time-to-create-the-whole-analysis-dataframe-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Time to create the whole analysis dataframe</a></span></li><li><span><a href="#Finding-out-where-the-time-is-being-taken" data-toc-modified-id="Finding-out-where-the-time-is-being-taken-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Finding out where the time is being taken</a></span></li></ul></li><li><span><a href="#Creating-Dataframes" data-toc-modified-id="Creating-Dataframes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creating Dataframes</a></span><ul class="toc-item"><li><span><a href="#Timing-the-difference" data-toc-modified-id="Timing-the-difference-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Timing the difference</a></span></li><li><span><a href="#List-of-dfs-vs-concatenating-dataframes" data-toc-modified-id="List-of-dfs-vs-concatenating-dataframes-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>List of dfs vs concatenating dataframes</a></span></li><li><span><a href="#List-of-Lists-to-DataFrame" data-toc-modified-id="List-of-Lists-to-DataFrame-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>List of Lists to DataFrame</a></span></li></ul></li><li><span><a href="#Comparing-final-outputs" data-toc-modified-id="Comparing-final-outputs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Comparing final outputs</a></span><ul class="toc-item"><li><span><a href="#list-of-lists-to-df" data-toc-modified-id="list-of-lists-to-df-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>list of lists to df</a></span></li></ul></li></ul></div>

# Imports

In [5]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [15]:
import pandas as pd
import time

In [8]:
from wordle_solver import wordle_functions as wf
from wordle_solver import file_imports
from definitions import ROOT_DIR

In [9]:
file_path = os.path.join(ROOT_DIR, "word_lists", "five_letter_words_no_caps.json")
csv_path = os.path.join(ROOT_DIR, "word_lists", "starter_words.csv")
guesses_path = os.path.join(ROOT_DIR, "word_lists", "wordle_candidates_nyt.json")
solutions_path = os.path.join(ROOT_DIR, "word_lists", "wordle_solutions_nyt.json")

In [13]:
five_letter_words = file_imports.import_json_to_list(file_path)
allowed_guesses = file_imports.import_json_to_list(guesses_path)["words"]
potential_solutions = file_imports.import_json_to_list(solutions_path)["words"]
total_guesses = allowed_guesses + potential_solutions

potential_solutions[:10]

['cigar',
 'rebut',
 'sissy',
 'humph',
 'awake',
 'blush',
 'focal',
 'evade',
 'naval',
 'serve']

In [ ]:
def test_word_guesser(all_words, potential_words):
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        guess_df = wf.create_guess_df(i, expected_words_left, sum_of_groups, potential_solution)
        first_guess_df = pd.concat([first_guess_df, guess_df])
    first_guess_df = first_guess_df.sort_values(by=["NumberOfGroups","PotentialSolution"], ascending=[False, False])
    return first_guess_df

# Feedback
Evaluating how long it takes to provide "feedback" or the colour hints returned using the return_corrected_wordle_clue function

In [83]:
test_guesses = total_guesses # ["metes"] #
test_solutions = potential_solutions

def test_feedback(guess, word_to_guess,  potential_words):
    start_time = time.time()
    feedback = wf.return_corrected_wordle_clue(guess, word_to_guess)
    solutions = [i for i in potential_words if
                      wf.return_corrected_wordle_clue(guess, i) == feedback]
    duration = time.time() - start_time
    print(f'feedback for {len([guess])} guesses for {len(potential_words)} solutions takes {duration}')
    print(f'feedbacck for 10000 guess would take {duration * 10000} seconds')
    

test_feedback("trace","story", test_solutions)

feedback for 1 guesses for 2309 solutions takes 0.016982078552246094
feedbacck for 10000 guess would take 169.82078552246094 seconds


## returning the clue as a string vs a list

In [84]:
def test_time_compute_wordle_colours_simple(all_words, potential_words):
    start_time = time.time()
    for i in all_words:
        for word in potential_words:
            wf.return_wordle_colours_list(i, word)

    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    pass

def test_time_compute_wordle_colours_simple_str(all_words, potential_words):
    start_time = time.time()
    for i in all_words:
        for word in potential_words:
            wf.return_wordle_colours_str(i, word)

    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    pass

test_guesses = total_guesses[:1000] # ["metes"] #
test_solutions = potential_solutions

test_time_compute_wordle_colours_simple(test_guesses, test_solutions)
test_time_compute_wordle_colours_simple_str(test_guesses, test_solutions)

1000 guesses for 2309 solutions takes 4.561352491378784
1000 guesses for 2309 solutions takes 4.4751927852630615


## Evaluating the time to correct the wordle feedback
This code uses the "return_corrected_wordle_clue" function to get feedback for all potential guesses against a list of potential solutions

In [24]:
def test_time_return_corrected_wordle_clue(all_words, potential_words):
    start_time = time.time()
    for i in all_words:
        for word in potential_words:
            feedback = wf.return_corrected_wordle_clue(i, word)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return i, word, feedback

test_guesses = total_guesses[:1000] 
test_solutions = potential_solutions

test_time_return_corrected_wordle_clue(test_guesses, test_solutions)

1000 guesses for 2309 solutions takes 10.608299970626831


('bocca', 'shave', 'BBBBY')

We can see that this process takes 10 seconds, rather than the 4.5 seconds the "simple" wordle feedback would take.

The compute_wordle_colours function attempts to make this process more efficient.

## Speeding up the feedback

In [85]:
def test_time_compute_wordle_colours(all_words, potential_words):
    start_time = time.time()
    for i in all_words:
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return i, potential_words[-1], guess_results[potential_words[-1]]

test_guesses = total_guesses[:1000] 
test_solutions = potential_solutions

test_time_compute_wordle_colours(test_guesses, test_solutions)

1000 guesses for 2309 solutions takes 7.214657545089722


('bocca', 'shave', 'BBBBY')

Using the "compute_wordle_colours_for_one_word" function speeds up the process

# Evaluating the next guess analysis

## Time to get feedback

Using a situation where the first guess was "trace" and the secret word was story, we can test how long it would take to calculate the optimal next guess.

To start with we can look at how long it would take to just calculate the feedback for each potential word against the remaining solutions

In [38]:
def test_time_compute_wordle_colours(all_words, potential_words):
    start_time = time.time()
    for i in all_words:
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return i, potential_words[-1], guess_results[potential_words[-1]]

feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]
print(feedback, test_solutions, len(test_solutions), len(test_guesses))
test_time_compute_wordle_colours(test_guesses, test_solutions)

YYBBB ['forth', 'first', 'motor', 'blurt', 'robot', 'story', 'girth', 'rusty', 'sport', 'worst', 'strut', 'burnt', 'stork', 'storm', 'worth', 'short', 'flirt', 'forty', 'burst', 'snort', 'shirt', 'spurt', 'birth', 'strip', 'north', 'right', 'dirty', 'mirth', 'rotor', 'roost', 'intro', 'skirt'] 32 12947
12947 guesses for 32 solutions takes 1.2387464046478271


('shave', 'skirt', 'GBBBB')

## Time to create the whole analysis dataframe

In [86]:
def test_time_word_guesser(all_words, potential_words):
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        guess_df = wf.create_guess_df(i, expected_words_left, sum_of_groups, potential_solution)
        first_guess_df = pd.concat([first_guess_df, guess_df])
    first_guess_df = first_guess_df.sort_values(by=["NumberOfGroups","PotentialSolution"], ascending=[False, False])
    
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return first_guess_df

feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]
test_df = test_time_word_guesser(test_guesses, test_solutions)
test_df.head()

12947 guesses for 32 solutions takes 53.348191022872925


,Guess,AverageRemainingSolutions,NumberOfGroups,WordsPerGroup,PotentialSolution
0,horst,1.8750,21,1.52381,0
0,shout,2.0625,21,1.52381,0
0,short,2.0000,20,1.60000,1
0,birsy,2.4375,20,1.60000,0
0,horis,2.3125,20,1.60000,0


It takes 53 seconds for the analysis dataset to be produced. If we were to do this for each guess - analysing performacne for all words would take an impossibly long time. 

Given we know it only takes 1.2 seconds to calculate the wordle feedback for this situation - the time must be coming from some of the other functions.

## Finding out where the time is being taken
By adding functions to the analysis code one by one and timing them we can identify where the time is being taken.

In [87]:
def test_time_word_guesser_indiv(all_words, potential_words):
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
        
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        guess_df = wf.create_guess_df(i, expected_words_left, sum_of_groups, potential_solution)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        guess_df = wf.create_guess_df(i, expected_words_left, sum_of_groups, potential_solution)
        first_guess_df = pd.concat([first_guess_df, guess_df])
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    
    
feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]
print(feedback, test_solutions, len(test_solutions), len(test_guesses))

test_df = test_time_word_guesser_indiv(test_guesses, test_solutions)

YYBBB ['forth', 'first', 'motor', 'blurt', 'robot', 'story', 'girth', 'rusty', 'sport', 'worst', 'strut', 'burnt', 'stork', 'storm', 'worth', 'short', 'flirt', 'forty', 'burst', 'snort', 'shirt', 'spurt', 'birth', 'strip', 'north', 'right', 'dirty', 'mirth', 'rotor', 'roost', 'intro', 'skirt'] 32 12947
12947 guesses for 32 solutions takes 1.175452470779419
12947 guesses for 32 solutions takes 1.3877387046813965
12947 guesses for 32 solutions takes 1.4031789302825928
12947 guesses for 32 solutions takes 1.4203708171844482
12947 guesses for 32 solutions takes 29.039681434631348
12947 guesses for 32 solutions takes 55.723016023635864


From this we can see that the time comes from creating a dataframe for each word - and then even more time is taken from appending to our analysis dataframe each time

# Creating Dataframes
Creating a dataframe adds a lot of time to the process.

It might be more efficient to work with lists. Below is the original function used to create a df of summary stats as well as alternate code that could store the same information as a list.

In [88]:
# This is the process currently being used 
def create_guess_df(starting_word, words_left, word_groups, possible_solution):
    df = pd.DataFrame()
    df["Guess"] = [starting_word]
    df["AverageRemainingSolutions"] = [words_left]
    df["NumberOfGroups"] = [len(word_groups)]
    df["WordsPerGroup"] = [sum(word_groups.values()) / len(word_groups)]
    df["PotentialSolution"] = [possible_solution]
    return df

def create_guess_list(starting_word, words_left, word_groups, possible_solution):
    start_time = time.time()
    NumberOfGroups = len(word_groups)
    WordsPerGroup = sum(word_groups.values()) / len(word_groups)
    output_list = [starting_word, words_left, NumberOfGroups, WordsPerGroup, possible_solution]
    duration = time.time() - start_time
    return output_list

## Timing the difference

In [90]:
def test_word_guesser(all_words, potential_words):
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        # creating a dataframe
        guess_df = wf.create_guess_df(i, expected_words_left, sum_of_groups, potential_solution)

    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return guess_df

def test_word_guesser_list(all_words, potential_words):
    start_time = time.time()
    first_guess_list = []
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        # creating a list instead of a dataframe
        guess_list = create_guess_list(i, expected_words_left, sum_of_groups, potential_solution)
        
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return guess_list

feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]

guess_df = test_word_guesser(test_guesses, test_solutions)
guess_list = test_word_guesser_list(test_guesses, test_solutions)

12947 guesses for 32 solutions takes 30.23156762123108
12947 guesses for 32 solutions takes 1.4954402446746826


Creating a list instead of a dataframe speeds up the process immeasurably. However, we still want to create a dataframe of the outputs at some point

In [91]:
# this is the output from the dataframe process
guess_df.head()

,Guess,AverageRemainingSolutions,NumberOfGroups,WordsPerGroup,PotentialSolution
0,shave,7.5,5,6.4,0


In [92]:
# this is the output after converting the list output to a dataframe
column_names = ["Guess", "AverageRemainingSolutions", "NumberOfGroups", "WordsPerGroup", "PotentialSolution"]
pd.DataFrame([guess_list], columns =column_names)

,Guess,AverageRemainingSolutions,NumberOfGroups,WordsPerGroup,PotentialSolution
0,shave,7.5,5,6.4,0


In [93]:
def test_word_guesser_list(all_words, potential_words):
    start_time = time.time()
    first_guess_list = []
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        # creating a list instead of a dataframe
        guess_list = create_guess_list(i, expected_words_left, sum_of_groups, potential_solution)
        #creating a dataframe
        guess_df = pd.DataFrame([guess_list], columns =column_names)
        
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return guess_df

feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]

guess_df = test_word_guesser(test_guesses, test_solutions)
guess_list = test_word_guesser_list(test_guesses, test_solutions)

12947 guesses for 32 solutions takes 27.95076632499695
12947 guesses for 32 solutions takes 9.644094228744507


creating a dataframe this way is much quicker

## List of dfs vs concatenating dataframes
One way to speed up this process would be to create a list of dataframes and then concatenat them all together once - rather than concatenating the dataframe each time

In [98]:
def test_word_guesser_append_dfs(all_words, potential_words):
    start_time = time.time()
    first_guess_df = pd.DataFrame()
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        # creating a dataframe
        guess_list = create_guess_list(i, expected_words_left, sum_of_groups, potential_solution)
        guess_df = pd.DataFrame([guess_list], columns =column_names)
        
        # appending dataframes 
        first_guess_df = pd.concat([first_guess_df, guess_df], axis=0)

    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return first_guess_df

def test_word_guesser_list_of_dfs(all_words, potential_words):
    start_time = time.time()
    total_guess_list = []
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        # creating a list instead of a dataframe
        guess_list = create_guess_list(i, expected_words_left, sum_of_groups, potential_solution)
        guess_df = pd.DataFrame([guess_list], columns =column_names)
        
        #append to a list of dataframes
        total_guess_list.append(guess_df)
    
    #concatenate the list of dataframes to create a final dataframe
    first_guess_df = pd.concat(total_guess_list, axis=0)
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return first_guess_df

feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]

guess_df = test_word_guesser_append_dfs(test_guesses, test_solutions)
guess_list_dfs = test_word_guesser_list_of_dfs(test_guesses, test_solutions)

12947 guesses for 32 solutions takes 32.04566240310669
12947 guesses for 32 solutions takes 15.005730628967285


Concatenating the dataframe each time takes almost twice as long as creating a list of dataframes and concatenating them once

## List of Lists to DataFrame

In [99]:
def test_word_guesser_list_of_lists(all_words, potential_words):
    start_time = time.time()
    total_guess_list = []
    column_names = ["Guess", "AverageRemainingSolutions", "NumberOfGroups", "WordsPerGroup", "PotentialSolution"]
    for i in all_words: # ["raise","crane"]: #
        guess_results = wf.compute_wordle_colours_for_one_word(i, potential_words)
        sum_of_groups = wf.sum_groups_of_hints(guess_results)
        expected_words_left = wf.average_solutions_left_after_guess(sum_of_groups)
        potential_solution = wf.is_word_in_list(i, potential_words)
        # create list of results
        guess_list = create_guess_list(i, expected_words_left, sum_of_groups, potential_solution)
        # append to list of list of results
        total_guess_list.append(guess_list)
    
    # create dataframe from list of lists 
    first_guess_df = pd.DataFrame(total_guess_list, columns =column_names)
    first_guess_df = first_guess_df.sort_values(by=["NumberOfGroups","PotentialSolution"], ascending=[False, False])
    duration = time.time() - start_time
    print(f'{len(all_words)} guesses for {len(potential_words)} solutions takes {duration}')
    return first_guess_df

feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]

# this is hopefully the most optimal solution
all_guess_df = test_word_guesser_list_of_lists(test_guesses, test_solutions)

12947 guesses for 32 solutions takes 13.98380160331726
12947 guesses for 32 solutions takes 1.5815777778625488


This now takes 1.5 seconds!

# Comparing final outputs

In [100]:
feedback = wf.return_corrected_wordle_clue("trace", "story")
test_guesses = total_guesses
test_solutions = [i for i in potential_solutions if wf.return_corrected_wordle_clue("trace", i) == feedback]
test_df = test_time_word_guesser(test_guesses, test_solutions)
test_df.head()

12947 guesses for 32 solutions takes 54.54368448257446


,Guess,AverageRemainingSolutions,NumberOfGroups,WordsPerGroup,PotentialSolution
0,horst,1.8750,21,1.52381,0
0,shout,2.0625,21,1.52381,0
0,short,2.0000,20,1.60000,1
0,birsy,2.4375,20,1.60000,0
0,horis,2.3125,20,1.60000,0


In [101]:
all_guess_df = test_word_guesser_list_of_lists(test_guesses, test_solutions)
all_guess_df.head()

12947 guesses for 32 solutions takes 1.4535894393920898


,Guess,AverageRemainingSolutions,NumberOfGroups,WordsPerGroup,PotentialSolution
4161,horst,1.8750,21,1.52381,0
12484,shout,2.0625,21,1.52381,0
11865,short,2.0000,20,1.60000,1
912,birsy,2.4375,20,1.60000,0
4157,horis,2.3125,20,1.60000,0
